In [5]:
from ase.io import read
from npl.core import Nanoparticle as NP
from npl.descriptors import EnvironmentalTopologies as ETOP
import numpy as np
from numba import njit

system = read("/home/riccardo/Scientific_Projects/PtAu_bigNps/300K/10_perc/run2/glo002.xyz")
system = NP.from_atoms(system)

bond_matrix = system.get_bond_matrix()
occupation_matrix = system._occupation_matrix

@njit
def get_top_descriptors(occupation_matrix, bond_matrix):
    feature_vector = np.zeros(29)
    for symbol, index_bonds  in zip(occupation_matrix[0], bond_matrix):
        a_bonds, b_bonds = index_bonds
        cn = int(sum(index_bonds))
        if symbol == 1:
            feature_vector[0] += a_bonds/2
            feature_vector[2] += b_bonds/2
            feature_vector[cn+3] += 1
        else:
            feature_vector[2] += a_bonds/2
            feature_vector[1] += b_bonds/2
            feature_vector[cn+3+13] += 1



In [10]:
system._get

array([79, 79, 78, ..., 78, 78, 78])

In [8]:
%timeit get_top_descriptors(system._occupation_matrix, system.get_bond_matrix())

3.02 ms ± 240 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


20.8 µs ± 471 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [4]:
%timeit ETOP().update(system)

2.72 ms ± 120 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
class ExchangeMove:
    def __init__(self, system):
        self.occupation_matrix = system._occupation_matrix
        self.a = np.where(self.occupation_matrix==1)[1]
        self.b = np.where(self.occupation_matrix==0)[1]

    def exchange_move(self):
        swap1 = np.random.choice(self.a)
        swap2 = np.random.choice(self.b)

        self.a[swap1], self.b[swap2] = self.a[swap2], self.b[swap1]

        self.occupation_matrix[0,swap1], self.occupation_matrix[1,swap1] = self.occupation_matrix[1,swap1], self.occupation_matrix[0,swap1]
        self.occupation_matrix[0,swap2], self.occupation_matrix[1,swap2] = self.occupation_matrix[1,swap2], self.occupation_matrix[0,swap2]

ex = ExchangeMove(system)
ex.exchange_move()

In [39]:
for _ in range(20000):
    ex.exchange_move()
    bond_matrix = system.get_bond_matrix()
    get_top_descriptors(system._occupation_matrix, bond_matrix)

In [20]:
np.where(system._occupation_matrix==1)[1]

array([   0,    1,    3, ..., 1460, 1461, 1462])

In [63]:
@njit
def dio(ciao, ciao2):
    ciao.dot(ciao2) 

%timeit dio(system._connectivity_matrix, system._occupation_matrix.T)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
- Resolution failure for literal arguments:
No implementation of function Function(<function array_dot at 0x7fac07f6e940>) found for signature:

 >>> array_dot(array(int16, 2d, C), array(float64, 2d, F))

There are 2 candidate implementations:
      - Of which 2 did not match due to:
      Overload in function 'array_dot': File: numba/np/arrayobj.py: Line 5526.
        With argument(s): '(array(int16, 2d, C), array(float64, 2d, F))':
       Rejected as the implementation raised a specific error:
         TypingError: Failed in nopython mode pipeline (step: nopython frontend)
       No implementation of function Function(<function dot at 0x7fac70599790>) found for signature:

        >>> dot(array(int16, 2d, C), array(float64, 2d, F))

       There are 4 candidate implementations:
             - Of which 4 did not match due to:
             Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: 

In [55]:
system._connectivity_matrix * system._occupation_matrix[1]

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [59]:
system._connectivity_matrix.dot(system._occupation_matrix.T)

array([[6., 3.],
       [4., 5.],
       [4., 5.],
       ...,
       [5., 4.],
       [7., 2.],
       [7., 2.]])